# Set up

In [1]:
!python -V

Python 3.10.9


In [2]:
import pandas as pd

In [4]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [68]:
def read_dataframe(filename):
    if filename.endswith('.csv'):
        df = pd.read_csv(filename)

        df.lpep_dropoff_datetime = pd.to_datetime(df.tpep_dropoff_datetime)
        df.lpep_pickup_datetime = pd.to_datetime(df.tpep_pickup_datetime)
    elif filename.endswith('.parquet'):
        df = pd.read_parquet(filename)

#     df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
#     df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

#     df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

In [69]:
df_train = read_dataframe('./data/yellow_tripdata_2022-01.parquet')
df_val = read_dataframe('./data/yellow_tripdata_2022-02.parquet')

In [70]:
len(df_train), len(df_val)

(2463931, 2979431)

# Q1 - how many columns in January data?

In [71]:
df_train.shape[1]

19

# Q2 - Compute duration variable and find standard deviation

In [72]:
def add_duration_column(df):
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df['duration'] = df['duration'].apply(lambda td: td.total_seconds() / 60)
#     df = df[(df['duration'] >= 1) & (df['duration'] <= 60)]
    return df

df_train = add_duration_column(df_train)
df_val = add_duration_column(df_val)


In [73]:
df_train['duration'].std()

46.44530513776499

# Q3 - dropping outliers; find fraction of records left

In [74]:
def remove_outliers(df):
    df = df[(df['duration'] >= 1) & (df['duration'] <= 60)]
    return df

df_train = remove_outliers(df_train)
df_val = remove_outliers(df_val)

In [75]:
len(df_train) / 2463931

0.9827547930522406

# Q4 - one hot encoding; find dimensionality of new matrix

In [76]:
categorical = ['PULocationID', 'DOLocationID']
numerical = [] # ['trip_distance']

dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [78]:
X_train.shape

(2421440, 515)

# Q5 - Training model; find RMSE on training set

In [79]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [80]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_train)

mean_squared_error(y_train, y_pred, squared=False)

6.986190836477672

# Q6 - evaluate model on validation set

In [81]:
y_pred = lr.predict(X_val)

mean_squared_error(y_val, y_pred, squared=False)

7.78640879016696